In [ ]:
# === SEKCJA 1: IMPORTY I KONFIGURACJA ===
import pandas as pd
import mlflow
from pycaret.classification import setup, pull, compare_models, create_model, tune_model, plot_model, finalize_model, save_model
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Konfiguracja MLflow
MLFLOW_EXPERIMENT_NAME = 'Investoro_District_Classification' # Nowa, bardziej adekwatna nazwa
mlflow.set_tracking_uri("http://localhost:5000") # Upewnij się, że to poprawny adres
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)
print(f"MLflow ustawiony. Eksperyment: '{MLFLOW_EXPERIMENT_NAME}'")

# Wczytanie danych
df_original = pd.read_csv('data.csv', sep=',')
print(f"Wczytano dane. Kształt: {df_original.shape}")

In [ ]:
df_original .head(10)

In [ ]:
df_original.info()

In [ ]:
df_original [df_original .duplicated()]

In [ ]:
df_original [df_original .duplicated()]

In [ ]:
df_corr_temp = df_original.copy()
if pd.api.types.is_string_dtype(df_corr_temp['BuiltYear']):
    df_corr_temp['BuiltYear_Num'] = pd.to_datetime(df_corr_temp['BuiltYear'], format='%Y', errors='coerce').dt.year
elif pd.api.types.is_datetime64_any_dtype(df_corr_temp['BuiltYear']):
     df_corr_temp['BuiltYear_Num'] = df_corr_temp['BuiltYear'].dt.year
else:
    df_corr_temp['BuiltYear_Num'] = pd.to_numeric(df_corr_temp['BuiltYear'], errors='coerce') # Ostateczna próba

cols_for_corr = ['Area', 'Price', 'BuiltYear_Num', 'Floor', 'Floors','Location', 'CountyNumber', 'CommunityNumber',
                   'RegionNumber','KindNumber']
# Upewnij się, że wszystkie kolumny istnieją i są numeryczne
valid_cols_for_corr = [col for col in cols_for_corr if col in df_corr_temp.columns and pd.api.types.is_numeric_dtype(df_corr_temp[col])]
correlation_matrix = df_corr_temp[valid_cols_for_corr].corr()

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Price')

In [ ]:
df_original .isnull().sum()

In [ ]:
# === SEKCJA 2: CZYSZCZENIE DANYCH ===
print(f"Oryginalny rozmiar danych: {df_original.shape}")
df_cleaned = df_original.copy()

# Krok 1: Usunięcie kluczowych braków danych
df_cleaned.dropna(subset=['Area', 'Price', 'Location', 'Description'], inplace=True)
print(f"Rozmiar po dropna (Area, Price, Location, Description): {df_cleaned.shape}")

# Krok 2: Usunięcie outlierów na podstawie ceny (IQR)
Q1_price = df_cleaned["Price"].quantile(0.25)
Q3_price = df_cleaned["Price"].quantile(0.75)
IQR_price = Q3_price - Q1_price
df_cleaned = df_cleaned[~((df_cleaned["Price"] < (Q1_price - 1.5 * IQR_price)) | (df_cleaned["Price"] > (Q3_price + 1.5 * IQR_price)))]
print(f"Rozmiar po usunięciu outlierów z Price: {df_cleaned.shape}")

# Krok 3: Usunięcie outlierów na podstawie powierzchni (IQR)
Q1_area = df_cleaned["Area"].quantile(0.25)
Q3_area = df_cleaned["Area"].quantile(0.75)
IQR_area = Q3_area - Q1_area
df_cleaned = df_cleaned[~((df_cleaned["Area"] < (Q1_area - 1.5 * IQR_area)) | (df_cleaned["Area"] > (Q3_area + 1.5 * IQR_area)))]
print(f"Rozmiar po usunięciu outlierów z Area: {df_cleaned.shape}")

# Krok 4: Konwersja daty
df_cleaned['BuiltYear'] = pd.to_datetime(df_cleaned['BuiltYear'], format='%Y', errors='coerce')

print("\nCzyszczenie danych zakończone.")

In [ ]:
print("Informacje o df_cleaned po wszystkich krokach czyszczenia:")
df_cleaned.info()
print("\nBraki danych w df_cleaned (%):")
display(df_cleaned.isnull().sum() / len(df_cleaned) * 100)
print("\nPierwsze wiersze df_cleaned:")
display(df_cleaned.head())

In [ ]:
# === SEKCJA 3: INŻYNIERIA CECH (NA CAŁYM ZBIORZE) ===
df_features = df_cleaned.copy()

# --- Krok 3a: Ekstrakcja Dzielnicy ---
def extract_district(location_str):
    if not isinstance(location_str, str): return np.nan
    parts = [part.strip() for part in location_str.split(',')]
    if len(parts) >= 3: return f"{parts[1]}, {parts[2]}" # Miasto, Dzielnica
    elif len(parts) == 2: return parts[1] # Samo miasto
    else: return parts[0]

df_features['District'] = df_features['Location'].apply(extract_district)
df_features.dropna(subset=['District'], inplace=True)
print(f"Dodano kolumnę 'District'. Liczba unikalnych wartości: {df_features['District'].nunique()}")

# --- Krok 3b: Tworzenie cech TF-IDF z opisu ---
vectorizer = TfidfVectorizer(max_features=100, ngram_range=(1, 2), min_df=5, max_df=0.95)

# Używamy fit_transform na całym zbiorze, bo to unsupervised.
# Indeksy muszą się zgadzać, dlatego resetujemy je przed i po.
df_features.reset_index(drop=True, inplace=True)
tfidf_features = vectorizer.fit_transform(df_features['Description'].fillna(''))
feature_names = vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=['des_tfidf_' + name for name in feature_names])

# Łączymy wszystko w jeden finalny DataFrame
df_final = pd.concat([df_features, tfidf_df], axis=1)
print(f"Dodano cechy TF-IDF. Finalny kształt zbioru do modelowania: {df_final.shape}")

In [ ]:
print(f"Rozmiar df_cleaned przed podziałem na train/holdout: {df_final.shape}")
train_df = df_final.sample(frac=0.9, random_state=42)
holdout_df = df_final.drop(train_df.index)

print(f"Rozmiar zbioru treningowego (train_df): {train_df.shape}")
print(f"Rozmiar zbioru holdout (holdout_df): {holdout_df.shape}")

In [ ]:
import os
# Utwórz dedykowany katalog dla tego testu, jeśli nie istnieje
current_directory = os.getcwd() 
local_mlruns_path = os.path.join(current_directory, "mlruns_DIRECT_LOCAL_TEST") 

if not os.path.exists(local_mlruns_path):
    os.makedirs(local_mlruns_path)
    print(f"Utworzono katalog: {local_mlruns_path}")
else:
    print(f"Katalog już istnieje: {local_mlruns_path}")

absolute_mlruns_path = os.path.abspath(local_mlruns_path)
tracking_uri = f"file:///{absolute_mlruns_path.replace(os.sep, '/')}"
mlflow.set_tracking_uri(tracking_uri)

print(f"MLflow tracking URI ustawione na: {mlflow.get_tracking_uri()}")

# MLFLOW_EXPERIMENT_NAME powinno być zdefiniowane wcześniej w Twoim notebooku
# np. MLFLOW_EXPERIMENT_NAME = 'Investoro_Ceny'

try:
    # Sprawdź, czy eksperyment istnieje
    experiment = mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
    if experiment is None:
        experiment_id = mlflow.create_experiment(MLFLOW_EXPERIMENT_NAME)
        print(f"Utworzono nowy eksperyment MLflow: '{MLFLOW_EXPERIMENT_NAME}' o ID: {experiment_id}")
    else:
        experiment_id = experiment.experiment_id
        print(f"Znaleziono istniejący eksperyment: '{MLFLOW_EXPERIMENT_NAME}' o ID: {experiment_id}")
    
    # Ustaw eksperyment jako aktywny
    mlflow.set_experiment(experiment_name=MLFLOW_EXPERIMENT_NAME)
    print(f"Aktywny eksperyment MLflow ustawiony na: '{MLFLOW_EXPERIMENT_NAME}'")

except Exception as e:
    print(f"Błąd podczas ustawiania/tworzenia eksperymentu MLflow: {e}")
    import traceback
    print(traceback.format_exc())


In [ ]:
import pandas as pd
import numpy as np

print(f"Oryginalny kształt danych: {train_df_processed.shape}")

# Funkcja do ekstrakcji dzielnicy z kolumny 'Location'
def extract_district(location_str):
    if not isinstance(location_str, str):
        return np.nan  # Zwróć NaN, jeśli wartość nie jest stringiem
    
    parts = [part.strip() for part in location_str.split(',')]
    
    # === Logika ekstrakcji (można ją dostosować) ===
    # Zakładamy, że interesuje nas miasto i dzielnica, jeśli są dostępne
    # Przykład: 'Mazowieckie, Warszawa, Mokotów' -> 'Warszawa, Mokotów'
    
    if len(parts) >= 3:
        # Bierzemy miasto i dzielnicę
        return f"{parts[1]}, {parts[2]}"
    elif len(parts) == 2:
        # Jeśli jest tylko województwo i miasto, bierzemy miasto
        return parts[1]
    elif len(parts) == 1:
        # Jeśli jest tylko jeden człon, bierzemy go
        return parts[0]
    else:
        return np.nan

# Stwórz nową kolumnę 'District' w swoim głównym DataFrame
# Użyjemy .copy(), aby uniknąć ostrzeżeń SettingWithCopyWarning
train_df_processed = train_df_processed.copy()
train_df_processed['District'] = train_df_processed['Location'].apply(extract_district)

# Usuń wiersze, gdzie nie udało się wyodrębnić dzielnicy
train_df_processed.dropna(subset=['District'], inplace=True)

print(f"Kształt danych po dodaniu kolumny 'District': {train_df_processed.shape}")
print("\nPrzykładowe wyekstrahowane dzielnice:")
print(train_df_processed[['Location', 'District']].head(10))

print(f"\nLiczba unikalnych dzielnic do klasyfikacji: {train_df_processed['District'].nunique()}")

In [ ]:
import pandas as pd
from IPython.display import display

# Ustawiamy próg, ile minimalnie razy musi wystąpić dana lokalizacja
# 2 to absolutne minimum, aby naprawić błąd. Wartości 3-5 mogą dać stabilniejszy model.
MIN_SAMPLES_PER_LOCATION = 3 

# Policz, ile razy występuje każda unikalna lokalizacja
location_counts = train_df_processed['Location'].value_counts()

# Zidentyfikuj lokalizacje, które występują rzadziej niż nasz próg
locations_to_remove = location_counts[location_counts < MIN_SAMPLES_PER_LOCATION].index

# Stwórz nową ramkę danych, usuwając wiersze z rzadkimi lokalizacjami
df_filtered = train_df_processed[~train_df_processed['Location'].isin(locations_to_remove)]

# Wyświetl informację, ile danych zostało usuniętych
print(f"Oryginalna liczba wierszy: {len(train_df_processed)}")
print(f"Liczba usuniętych rzadkich lokalizacji: {len(locations_to_remove)}")
print(f"Liczba wierszy po odfiltrowaniu: {len(df_filtered)}")
print("-" * 30)

In [ ]:
import mlflow

# Sprawdź, czy jest aktywny run i zamknij go, jeśli tak
if mlflow.active_run():
    mlflow.end_run()
    print("Aktywny run został zamknięty.")
else:
    print("Nie znaleziono aktywnego runu do zamknięcia.")

In [ ]:
# === SEKCJA 4: FILTROWANIE, PODZIAŁ I SETUP PYCARET ===

# --- Krok 4a: Filtrowanie rzadkich dzielnic ---
MIN_SAMPLES_PER_DISTRICT = 3
district_counts = df_final['District'].value_counts()
districts_to_remove = district_counts[district_counts < MIN_SAMPLES_PER_DISTRICT].index
df_model_ready = df_final[~df_final['District'].isin(districts_to_remove)]
print(f"Usunięto {len(districts_to_remove)} rzadkich dzielnic. Pozostało {len(df_model_ready)} wierszy.")
print(f"Liczba klas do predykcji: {df_model_ready['District'].nunique()}")

# --- Krok 4b: Podział na zbiór treningowy i testowy (holdout) ---
data_train = df_model_ready.sample(frac=0.9, random_state=1122)
data_test = df_model_ready.drop(data_train.index)
print(f"Podział danych -> Treningowe: {data_train.shape}, Testowe: {data_test.shape}")

# --- Krok 4c: Definicja list cech (teraz jest to proste i jednoznaczne) ---
numeric_features = [c for c in data_train.columns if c.startswith('des_tfidf_')] + ['Area', 'NumberOfRooms', 'Floor', 'Floors']
categorical_features = ['BuildingType', 'BuildingCondition', 'TypeOfMarket', 'OwnerType', 'Type', 'OfferFrom', 'VoivodeshipNumber', 'CountyNumber']
date_features = ['BuiltYear']

# Usuń z list cechy, których może nie być
numeric_features = [f for f in numeric_features if f in data_train.columns]
categorical_features = [f for f in categorical_features if f in data_train.columns]

# --- Krok 4d: Setup PyCaret ---
# Upewnij się, że nie ma aktywnego runu
if mlflow.active_run():
    mlflow.end_run()

with mlflow.start_run(run_name="District_Classification_Run") as run:
    print(f"\nRozpoczynam setup PyCaret w runie MLflow: {run.info.run_id}")
    
    # Lista ignorowanych cech - usuwamy wszystko, co nie jest cechą lub celem
    ignore_features = ['Location', 'District', 'Description', 'Title', 'Price'] + [c for c in data_train.columns if c.startswith('Date') and c != 'BuiltYear']
    
    s = setup(
        data=data_train,
        test_data=data_test, # Przekazujemy zbiór testowy do PyCaret
        target='District',
        session_id=1122,
        log_experiment=True, # PyCaret zaloguje do naszego aktywnego runu
        
        numeric_features=numeric_features,
        categorical_features=categorical_features,
        date_features=date_features,
        
        ignore_features=ignore_features,
        
        # Zabezpieczenia
        html=False,
        verbose=False
    )
    print("Setup PyCaret zakończony.")

In [ ]:
# === SEKCJA 5: PORÓWNANIE I ANALIZA MODELI ===

# Uruchom w ramach tego samego bloku 'with' z poprzedniej komórki lub osobno
# (PyCaret zapamięta sesję)
print("\nRozpoczynam porównywanie modeli...")
best_model = compare_models(sort='F1', n_select=1) # n_select=1 dla pewności, że dostajemy jeden model
print("\nPorównanie modeli zakończone.")

print("\nTabela z metrykami dla wszystkich modeli:")
all_models_metrics = pull()
display(all_models_metrics)

print(f"\nNajlepszy zidentyfikowany model: {best_model}")

In [ ]:
print("Metryki dla najlepszego modelu:")
best_model_metrics = all_models_metrics.head(1)
display(best_model_metrics)

In [ ]:
# === SEKCJA 6: TUNING I FINALIZACJA ===

print("\nRozpoczynam tuning najlepszego modelu...")
tuned_model = tune_model(best_model, n_iter=25, sort='F1')
print("\nTuning zakończony. Wyniki po tuningu:")
display(pull())

print("\nFinalizowanie modelu (trening na całym zbiorze danych)...")
final_model = finalize_model(tuned_model)
print(f"Model sfinalizowany: {final_model}")

# Zapisz finalny model
save_model(final_model, 'district_classifier_final_model')
print("\nFinalny model został zapisany do pliku.")

In [ ]:
# Porównaj wyniki przed i po tuningu
print("\n--- Wyniki po tuningu ---")
tuned_metrics = pull()
display(tuned_metrics)

In [ ]:
print("\n--- Najlepszy model przed tuningiem ---")
display(all_models_metrics.head(1))